In [1]:
import os, sys, time
import pickle
import pandas as pd
import numpy as np

from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
business = pd.read_csv('business_attributes_5.11.csv')
business.head()

C:\Users\mm4me\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (34,35,43,54,57) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,business_id,total_counts,45_counts,text,text_friendly,text_reasonable,text_local,text_comfortable,text_easy,...,smoking,wheelchairaccessible,wifi,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,0,--1UhMGODdWsrMastO9DZw,24,19.0,Came here for lunch last week and was pleasant...,0.291667,0.041667,0.000000,0.000000,0.083333,...,NaN,NaN,NaN,11:0-20:0,11:0-20:0,11:0-20:0,11:0-20:0,11:0-20:0,11:0-4:0,NaN
1,1,--6MefnULPED_I942VcFNA,39,17.0,Johns Chinese BBQ ruined Char Siu Pork for me....,0.051282,0.000000,0.025641,0.000000,0.000000,...,NaN,True,no,11:0-22:30,11:0-22:30,11:0-22:30,11:0-22:30,11:0-22:30,11:0-22:30,11:0-22:30
2,2,--7zmmkVg-IMGaXbuVd0SQ,54,39.0,"Great beer, great place, excellent service fro...",0.166667,0.000000,0.129630,0.055556,0.111111,...,NaN,True,NaN,16:0-22:0,16:0-22:0,14:0-22:0,14:0-22:0,12:0-23:0,12:0-23:0,12:0-21:0
3,3,--8LPVSo5i0Oo61X01sV9A,4,2.0,Dr. LaCognata is great. He is a great listene...,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,8:30-16:30,8:30-16:30,8:30-16:30,8:30-16:30,8:30-16:30,NaN,NaN
4,4,--9QQLMTbFzLJ_oT-ON3Xw,11,7.0,I've been going to this particular location fo...,0.000000,0.000000,0.000000,0.090909,0.090909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# drop the first column
business.drop(business.columns[0], axis=1, inplace=True)

In [4]:
business.price.isnull().sum()

81473

In [5]:
print(business.dtypes)

business_id                   object
total_counts                   int64
45_counts                    float64
text                          object
text_friendly                float64
text_reasonable              float64
text_local                   float64
text_comfortable             float64
text_easy                    float64
text_quick                   float64
1star_counts                 float64
2star_counts                 float64
3star_counts                 float64
4star_counts                 float64
5star_counts                 float64
address                       object
attributes                    object
categories                    object
city                          object
hours                         object
is_open                        int64
latitude                     float64
longitude                    float64
name                          object
neighborhood                  object
postal_code                   object
review_count                   int64
s

In [6]:
business_orig = pd.read_csv('business.csv')
business_orig.shape

(188593, 15)

In [7]:
business_orig.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,NaN,T2E 6L6,24,4.0,AB
1,NaN,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,NaN,89002,3,4.5,NV
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC
3,211 W Monroe St,NaN,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,NaN,1,33.449999,-112.076979,Geico Insurance,NaN,85003,8,1.5,AZ
4,2005 Alyth Place SE,{'BusinessAcceptsCreditCards': 'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,51.035591,-114.027366,Action Engine,NaN,T2H 0N5,4,2.0,AB


In [8]:
business_orig.dtypes

address          object
attributes       object
business_id      object
categories       object
city             object
hours            object
is_open           int64
latitude        float64
longitude       float64
name             object
neighborhood     object
postal_code      object
review_count      int64
stars           float64
state            object
dtype: object

#### Take business category column and one-hot encode

In [9]:
business_orig['categories_clean'] = business_orig['categories'].apply(lambda x: str(x).split(', '))

In [10]:
business_orig['categories_clean'][0]

['Tours', 'Breweries', 'Pizza', 'Restaurants', 'Food', 'Hotels & Travel']

In [11]:
print("The new column 'categories_clean' is a {} of {}.".format(type(business_orig['categories_clean'][0]),
                                                                type(business_orig['categories_clean'][0][0])))

The new column 'categories_clean' is a <class 'list'> of <class 'str'>.


#### Remove leading and trailing white space

In [12]:
list(i.lstrip().rstrip() for i in business_orig['categories_clean'][0])

['Tours', 'Breweries', 'Pizza', 'Restaurants', 'Food', 'Hotels & Travel']

In [13]:
def strip_space(lst):
    return [i.lstrip().rstrip() for i in lst]        

In [14]:
business_orig['categories_clean'] = business_orig['categories_clean'].apply(strip_space)

In [15]:
business_orig['categories_clean']

0         [Tours, Breweries, Pizza, Restaurants, Food, H...
1         [Chicken Wings, Burgers, Caterers, Street Vend...
2         [Breakfast & Brunch, Restaurants, French, Sand...
3                           [Insurance, Financial Services]
4         [Home & Garden, Nurseries & Gardening, Shoppin...
5                                      [Coffee & Tea, Food]
6                                          [Food, Bakeries]
7                                       [Restaurants, Thai]
8                                    [Mexican, Restaurants]
9                   [Flowers & Gifts, Gift Shops, Shopping]
10                                  [Restaurants, Japanese]
11                    [Cajun/Creole, Southern, Restaurants]
12        [Bars, Sports Bars, Dive Bars, Burgers, Nightl...
13         [Restaurants, Pakistani, Indian, Middle Eastern]
14                                 [Beauty & Spas, Barbers]
15                         [Delis, Restaurants, Sandwiches]
16        [Nightlife, Bars, American (Tr

#### Examine category frequency

In [16]:
# concatenate all
category = []
for i in business_orig['categories_clean']:
    category += i

category

['Tours',
 'Breweries',
 'Pizza',
 'Restaurants',
 'Food',
 'Hotels & Travel',
 'Chicken Wings',
 'Burgers',
 'Caterers',
 'Street Vendors',
 'Barbeque',
 'Food Trucks',
 'Food',
 'Restaurants',
 'Event Planning & Services',
 'Breakfast & Brunch',
 'Restaurants',
 'French',
 'Sandwiches',
 'Cafes',
 'Insurance',
 'Financial Services',
 'Home & Garden',
 'Nurseries & Gardening',
 'Shopping',
 'Local Services',
 'Automotive',
 'Electronics Repair',
 'Coffee & Tea',
 'Food',
 'Food',
 'Bakeries',
 'Restaurants',
 'Thai',
 'Mexican',
 'Restaurants',
 'Flowers & Gifts',
 'Gift Shops',
 'Shopping',
 'Restaurants',
 'Japanese',
 'Cajun/Creole',
 'Southern',
 'Restaurants',
 'Bars',
 'Sports Bars',
 'Dive Bars',
 'Burgers',
 'Nightlife',
 'Sandwiches',
 'Restaurants',
 'Restaurants',
 'Pakistani',
 'Indian',
 'Middle Eastern',
 'Beauty & Spas',
 'Barbers',
 'Delis',
 'Restaurants',
 'Sandwiches',
 'Nightlife',
 'Bars',
 'American (Traditional)',
 'Tapas/Small Plates',
 'Poutineries',
 'Supper 

In [17]:
Counter(category).most_common()

[('Restaurants', 57173),
 ('Shopping', 30231),
 ('Food', 27118),
 ('Beauty & Spas', 18967),
 ('Home Services', 18634),
 ('Health & Medical', 16157),
 ('Local Services', 12906),
 ('Automotive', 12656),
 ('Nightlife', 12438),
 ('Bars', 10853),
 ('Event Planning & Services', 9774),
 ('Active Life', 9119),
 ('Fashion', 7406),
 ('Coffee & Tea', 6936),
 ('Sandwiches', 6912),
 ('Hair Salons', 6825),
 ('Fast Food', 6812),
 ('American (Traditional)', 6659),
 ('Pizza', 6603),
 ('Home & Garden', 6020),
 ('Auto Repair', 5877),
 ('Hotels & Travel', 5808),
 ('Arts & Entertainment', 5794),
 ('Professional Services', 5620),
 ('Doctors', 5450),
 ('Real Estate', 5295),
 ('Burgers', 5126),
 ('Breakfast & Brunch', 5023),
 ('Nail Salons', 4839),
 ('Fitness & Instruction', 4559),
 ('Italian', 4550),
 ('Mexican', 4419),
 ('Specialty Food', 4304),
 ('Chinese', 4247),
 ('American (New)', 4230),
 ('Pets', 4001),
 ('Hair Removal', 3766),
 ('Bakeries', 3509),
 ('Dentists', 3436),
 ('Grocery', 3380),
 ('Skin Care'

In [18]:
counts = dict(zip(Counter(category).keys(), Counter(category).values()))
sorted(counts.items(), key=lambda x: x[1])

[('Eastern European', 1),
 ('Milkshake Bars', 1),
 ('Beach Bars', 1),
 ('Beach Volleyball', 1),
 ('Baguettes', 1),
 ('Guamanian', 1),
 ('Pensions', 1),
 ('Outdoor Movies', 1),
 ('Vocal Coach', 1),
 ('Bulgarian', 1),
 ('Hainan', 1),
 ('Rodeo', 1),
 ('Senegalese', 1),
 ('Halfway Houses', 1),
 ('Hybrid Car Repair', 1),
 ('Entertainment Law', 1),
 ('Customs Brokers', 1),
 ('Signature Cuisine', 1),
 ('Game Meat', 1),
 ('Props', 1),
 ('Bike Parking', 1),
 ('Czech/Slovakian', 1),
 ('Dialysis Clinics', 1),
 ('Rotisserie Chicken', 1),
 ('Minho', 1),
 ('Experiences', 1),
 ('Otologists', 1),
 ('Traditional Norwegian', 1),
 ('Sport Equipment Hire', 1),
 ('Ceremonial Clothing', 1),
 ('Tonkatsu', 1),
 ('Tempura', 1),
 ('Hang Gliding', 1),
 ('Beer Hall', 1),
 ('Churros', 1),
 ('Billing Services', 1),
 ('Water Suppliers', 1),
 ('Aviation Services', 1),
 ('Fishmonger', 1),
 ('Soba', 1),
 ('General Festivals', 1),
 ('Hearing Aids', 1),
 ('Udon', 1),
 ('Osteopaths', 1),
 ('Drive-Thru Bars', 1),
 ('Backsh

In [19]:
print("There are {} unique category values in the dataset.".format(len(set(category))))

There are 1306 unique category values in the dataset.


In [20]:
unique_elements, counts_elements = np.unique(category, return_counts=True)
print(list(zip(np.asarray((unique_elements, counts_elements)))))

[(array(['& Probates', '3D Printing', 'ATV Rentals/Tours', ..., 'Ziplining',
       'Zoos', 'nan'], dtype='<U35'),), (array(['79', '10', '53', ..., '7', '45', '541'], dtype='<U35'),)]


In [21]:
business_orig['categories_clean'][0]

['Tours', 'Breweries', 'Pizza', 'Restaurants', 'Food', 'Hotels & Travel']

In [22]:
# examine how many category terms there can be in a single business
business_orig['categories'].apply(lambda x: str(x).count(',') + 1).describe()

count    188593.000000
mean          3.921476
std           1.973396
min           1.000000
25%           2.000000
50%           3.000000
75%           5.000000
max          37.000000
Name: categories, dtype: float64

## Use categories from Yelp website
https://www.yelp.com/factsheet

- Restaurants
    - Sandwiches | Fast Food | American (Traditional) | Pizza
- Food (that is not 'Restaurants')
    - Coffee & Tea | Bakeries | Grocery | Cafes | Desserts
- Shopping
    - Fashion | Women's Clothing
- Etc
    - Hotels & Travel | Nightlife | Bars | Arts & Entertainment | Active Life
- Other
    - Event Planning & Services
    - Home Services | Local Services | Home & Garden
    - Beauty & Spas | Hair Salons | Nail Salons | Fitness & Instruction | Hair Removal | Skin Care
    - Health & Medical | Doctors | Dentists
    - Automotive | Auto Repair

 
NEED TO LOOK UP:
Professional Services
Real Estate
Pets
Education
Financial Services
Contractors

There are some businesses whose categories are null...

In [23]:
print(sum(business_orig.categories.isnull()))
business_orig[business_orig.categories.isnull()]

541


,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,categories_clean
288,"Adelaide Centre, 130 Adelaide St West Concours...","{'BikeParking': 'False', 'BusinessParking': ""{...",EBzr465prEffkpmE8Mk5AA,NaN,Toronto,"{'Monday': '7:0-19:0', 'Tuesday': '7:0-19:0', ...",1,43.649592,-79.383394,Polish'd Nail Bar,Financial District,M5H 3P5,7,2.5,ON,[nan]
603,7700 W Arrowhead Towne Ctr,NaN,CN3BLZwfG4eqZjvKrIZoAg,NaN,Glendale,NaN,1,33.642064,-112.225217,Fuzziwigs Candy Factory,NaN,85308,4,1.0,AZ,[nan]
1091,"20800 Center Ridge Rd, Ste 105",NaN,Y76jGd_f889OT4oz9pLgZA,NaN,Rocky River,NaN,1,41.462730,-81.850909,Center For Effective Living,NaN,44116,3,1.5,OH,[nan]
1179,"7250 N 16th St, Ste 105",NaN,yo8TQab_kYgAZm_DJkRjfA,NaN,Phoenix,NaN,1,33.543316,-112.047916,A L Financial,NaN,85020,12,1.0,AZ,[nan]
1404,371 Spadina Avenue,"{'BikeParking': 'True', 'BusinessParking': ""{'...",s_0zH6HExuWJeJjiZAtckA,NaN,Toronto,NaN,1,43.655797,-79.399046,CoCo Fresh Tea & Juice,Chinatown,M5T 2G3,32,3.0,ON,[nan]
1887,1158 Miles Rd,NaN,v7BcP2B6IGR59ZrY4QUaYA,NaN,Dallas,NaN,1,35.374975,-81.223308,Ray's,NaN,28034,3,1.0,NC,[nan]
2683,2929 E McKellips Rd,NaN,FXY6bj0GTWZoiA_uRng6xQ,NaN,Mesa,NaN,1,33.450629,-111.768880,Vista Grove Prep Academy,NaN,85213,3,2.5,AZ,[nan]
3802,"3415 W Glendale Ave, Ste 3",NaN,-J5SYRlH2hpv4uDs07lkqA,NaN,Phoenix,NaN,1,33.537164,-112.132926,Aerus Electrolux,NaN,85051,3,3.5,AZ,[nan]
4455,3055 Fremont St,NaN,nR6wsc5ld4bx6NTa0ogDwQ,NaN,Las Vegas,NaN,1,36.151870,-115.107819,Car Town U S A,Downtown,89104,4,1.0,NV,[nan]
4752,955 S Country Club Dr,NaN,IiJP9wxsnJBzUGLJKavHag,NaN,Mesa,NaN,1,33.397225,-111.839708,Hta Luxury and 4X4,NaN,85210,4,2.0,AZ,[nan]


In [24]:
business_orig_dropna = business_orig.dropna(axis=0, subset=['categories'])

In [25]:
# examine what categories are
cat_string = 'Hotels & Travel'
print(business_orig_dropna[business_orig_dropna["categories"].str.contains(cat_string)].shape[0])
business_orig_dropna[business_orig_dropna["categories"].str.contains(cat_string)]

5808


,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,categories_clean
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,NaN,T2E 6L6,24,4.0,AB,"[Tours, Breweries, Pizza, Restaurants, Food, H..."
43,3280 Kingston Road,NaN,zHwXoh40k86P0aiN1aix9Q,"Hotels, Hotels & Travel, Event Planning & Serv...",Toronto,NaN,1,43.733395,-79.224206,Super 8 by Wyndham Toronto East ON,Scarborough,M1M 1P8,3,2.0,ON,"[Hotels, Hotels & Travel, Event Planning & Ser..."
52,3105 Westwood Dr,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",VVtPNiqwtR_Os7J51mANGQ,"Motorcycle Rental, Active Life, Motorcycle Dea...",Las Vegas,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,36.133369,-115.178365,JAM Scooter Rental Of Las Vegas,NaN,89109,28,5.0,NV,"[Motorcycle Rental, Active Life, Motorcycle De..."
104,NaN,NaN,mKTq1T_IAplDpHUcMzOXkw,"Food Delivery Services, Transportation, Food, ...",Gastonia,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,35.267448,-81.187562,Tipsy Ryde,NaN,28054,3,3.5,NC,"[Food Delivery Services, Transportation, Food,..."
144,4742 Brecksville Rd,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",ahhMOgbZHCT_Xkhn4M49Dw,"Event Planning & Services, Hotels, Hotels & Tr...",Richfield,NaN,1,41.263562,-81.630059,Days Inn & Suites by Wyndham Richfield,NaN,44286,18,2.5,OH,"[Event Planning & Services, Hotels, Hotels & T..."
175,7180 Dean Martin Dr,NaN,arx-ZLLBY9RwOo_NahD9-g,"Transportation, Limos, Hotels & Travel",Las Vegas,NaN,1,36.059555,-115.182385,Executive Coach and Carriage,Southwest,89118,4,1.0,NV,"[Transportation, Limos, Hotels & Travel]"
182,3549 Industrial Rd,NaN,Tjjj9XSBc9Hr_RCGD6JURQ,"Hotels & Travel, Resorts",Las Vegas,NaN,1,36.125126,-115.177978,MGM Resorts International Contact Center,NaN,89109,4,1.0,NV,"[Hotels & Travel, Resorts]"
260,Esplanade,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",dYo5kYQg4PMC6a-Qy3gS7Q,"Event Planning & Services, Hotels & Travel, Ho...",Scarborough,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,54.276733,-0.399588,Crown Spa Hotel,NaN,YO11 2AG,3,4.0,NYK,"[Event Planning & Services, Hotels & Travel, H..."
298,101 Panther Hollow Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",eva56motCJcevOwKzyQO1g,"Cafes, Hotels & Travel, Swimming Pools, Travel...",Pittsburgh,"{'Monday': '6:0-23:0', 'Tuesday': '6:0-23:0', ...",1,40.437541,-79.948883,Schenley Park Visitors Center,Oakland,15213,12,5.0,PA,"[Cafes, Hotels & Travel, Swimming Pools, Trave..."
333,2050 Rue Saint-Denis,"{'BusinessAcceptsCreditCards': 'True', 'DogsAl...",m-QJ5htl3uuB0BnfBNSTYQ,"Hotels & Travel, Event Planning & Services, Be...",Montréal,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,45.516118,-73.566126,Hôtel Auberge Montréal Espace Confort,Ville-Marie,H2X 3K7,11,4.0,QC,"[Hotels & Travel, Event Planning & Services, B..."


All "Museums" fall under "Arts & Entertainment".

In [26]:
mask_arts = business_orig_dropna["categories"].str.contains("Arts & Entertainment")
mask_museum = business_orig_dropna["categories"].str.contains("Museum")

business_orig_dropna[~mask_arts & mask_museum]

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,categories_clean


All "Parks" fall under "Active Life" except for "RV Parks". No need for separate "Parks".

In [27]:
mask_parks = business_orig_dropna["categories"].str.contains("Parks")
mask_active = business_orig_dropna["categories"].str.contains("Active Life")

business_orig_dropna[mask_parks & ~mask_active]

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,categories_clean
2382,268 Lakeshore Rd,NaN,2zeiMDosX4HRPx7E5h_bEQ,"RV Parks, Campgrounds, Hotels & Travel",Boulder City,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,36.034521,-114.798592,Lake Mead RV Village - Echo Bay,NaN,89005,18,3.5,NV,"[RV Parks, Campgrounds, Hotels & Travel]"
3804,36408 N Black Canyon Hwy,NaN,OoUWfIQLcqzpL4Abo2UiTA,"Hotels & Travel, Resorts, RV Parks",Phoenix,"{'Monday': '7:0-18:0', 'Tuesday': '7:0-18:0', ...",1,33.823506,-112.149445,Pioneer RV Resort,NaN,85086,3,2.5,AZ,"[Hotels & Travel, Resorts, RV Parks]"
5504,3265 E Main St,{'BusinessAcceptsCreditCards': 'True'},t5MJzAFoXjV4PwDuN9IXEg,"RV Parks, Home Services, Real Estate, Campgrou...",Mesa,"{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ...",1,33.414631,-111.760278,Tiffany RV Park & Apartments,NaN,85213,4,3.0,AZ,"[RV Parks, Home Services, Real Estate, Campgro..."
5538,2929 E Main St,NaN,LUrRoI_F3wuZWU1QBf1QtQ,"Hotels & Travel, RV Parks, Health & Medical, R...",Mesa,NaN,1,33.411962,-111.767787,Palm Gardens,NaN,85213,3,3.5,AZ,"[Hotels & Travel, RV Parks, Health & Medical, ..."
5799,10950 W Union Hills Dr,NaN,259_yfBSoAMrhIb8PgBGtA,"Campgrounds, Hotels & Travel, RV Parks",Sun City,NaN,1,33.654308,-112.294858,Paradise RV Resort,NaN,85373,5,3.0,AZ,"[Campgrounds, Hotels & Travel, RV Parks]"
6878,2320 S Hardy Dr,NaN,bPegjJf-__OZFy914R5QgQ,"RV Parks, Local Services, Community Service/No...",Tempe,NaN,1,33.402673,-111.952717,Elks Lodge BPO E No 2251,NaN,85282,6,4.0,AZ,"[RV Parks, Local Services, Community Service/N..."
6982,10701 N 99th Ave,NaN,MKtF-UKMA6ZsGy3ue96FMg,"Hotels & Travel, Campgrounds, RV Parks, Home S...",Peoria,NaN,1,33.583160,-112.269533,Apollo Village RV Resort,NaN,85345,3,2.5,AZ,"[Hotels & Travel, Campgrounds, RV Parks, Home ..."
7610,130 Montée Monette,NaN,4mIyRDV4a1krLYC7Nfn_DQ,"Hotels, RV Parks, Campgrounds, Event Planning ...",Saint-Philippe,NaN,1,45.346753,-73.489582,Montréal Sud KOA,NaN,J0L 2K0,11,3.5,QC,"[Hotels, RV Parks, Campgrounds, Event Planning..."
8866,14050 W Van Buren St,{'BusinessAcceptsCreditCards': 'True'},n1TqzfsiqYpR2gkt0FQ69Q,"Local Services, Truck Rental, RV Parks, Hotels...",Goodyear,"{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ...",1,33.451167,-112.361357,Palm Gate RV & Self Storage,NaN,85338,6,5.0,AZ,"[Local Services, Truck Rental, RV Parks, Hotel..."
11529,6635 Boulder Hwy,NaN,fRQ4Rg_oCz6jV7XmccF1OQ,"Hotels & Travel, Campgrounds, RV Parks",Las Vegas,"{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ...",1,36.079956,-115.026951,Duck Creek RV Park,NaN,89122,18,3.0,NV,"[Hotels & Travel, Campgrounds, RV Parks]"


No need to separate out "Bars" from "Nightlife"

In [28]:
mask_nightlife = business_orig_dropna["categories"].str.contains("Nightlife")
mask_bars = business_orig_dropna["categories"].str.contains("Bars")
mask_nonebars = business_orig_dropna["categories"].str.contains("Sushi Bars|Juice Bars|Tapas Bars")

business_orig_dropna[~mask_nightlife & mask_bars & ~mask_nonebars]

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,categories_clean
4189,7320 E 6th Ave,"{'AcceptsInsurance': 'False', 'BikeParking': '...",sAMQU5CtyGiG4ZwRczN61g,"Oxygen Bars, IV Hydration, Beauty & Spas, Doct...",Scottsdale,"{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'...",1,33.499119,-111.923366,VIVA IV Therapy,NaN,85251,22,5.0,AZ,"[Oxygen Bars, IV Hydration, Beauty & Spas, Doc..."
5334,2000 Las Vegas Blvd S,"{'AcceptsInsurance': 'False', 'BikeParking': '...",VIxMt0NVS3O7M7dn6tmFpw,"Health & Medical, Oxygen Bars, Massage, Beauty...",Las Vegas,"{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",1,36.147247,-115.156031,Breathe Oxygen Bar,The Strip,89104,13,4.0,NV,"[Health & Medical, Oxygen Bars, Massage, Beaut..."
6351,2032 Queen Street E,"{'Alcohol': 'none', 'Ambience': ""{'romantic': ...",BNDhHX-TbsEw5QKJgHB_ag,"Restaurants, Milkshake Bars, American (Traditi...",Toronto,"{'Tuesday': '11:30-21:0', 'Wednesday': '11:30-...",0,43.670404,-79.298468,Shakes and Franks,The Beach,M4L 1J4,10,4.0,ON,"[Restaurants, Milkshake Bars, American (Tradit..."
38705,"9710 W Tropicana Ave, Ste 120","{'AcceptsInsurance': 'False', 'BikeParking': '...",CJ9ZY_b8WHDygndmq1g4zQ,"Reiki, Chiropractors, Health & Medical, Weight...",Las Vegas,"{'Monday': '10:30-20:30', 'Tuesday': '10:30-20...",1,36.100890,-115.304534,Wellness Body Wraps Health Studio,Spring Valley,89147,68,5.0,NV,"[Reiki, Chiropractors, Health & Medical, Weigh..."
42856,2545 Mosside Blvd,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",ABc-xuh5oKuoy9MWS1QOEw,"Massage, Pilates, Oxygen Bars, Eyelash Service...",Monroeville,"{'Monday': '10:0-20:0', 'Tuesday': '10:0-20:0'...",1,40.427416,-79.753251,Loughner Massage & Lash Boutique,NaN,15146,3,4.0,PA,"[Massage, Pilates, Oxygen Bars, Eyelash Servic..."
78379,"18589 N 59th Ave, Ste 109","{'AcceptsInsurance': 'False', 'BikeParking': '...",paySZ-XJBkAR7RD6La9Rnw,"Day Spas, Meditation Centers, Fitness & Instru...",Glendale,"{'Monday': '16:0-22:0', 'Tuesday': '10:0-22:0'...",1,33.653766,-112.184930,True REST Float Spa,NaN,85308,35,4.5,AZ,"[Day Spas, Meditation Centers, Fitness & Instr..."
86261,"2430 S Gilbert Rd, Ste B8","{'AcceptsInsurance': 'False', 'BusinessAccepts...",GSHUbRDDhFzqR2eTDiM3dg,"Float Spa, Oxygen Bars, Massage Therapy, Healt...",Chandler,"{'Monday': '15:0-22:0', 'Tuesday': '9:0-22:0',...",1,33.270758,-111.790651,True REST Float Spa,NaN,85286,69,5.0,AZ,"[Float Spa, Oxygen Bars, Massage Therapy, Heal..."
93356,NaN,"{'AcceptsInsurance': 'False', 'BikeParking': '...",c4PQ02A7r_2tnvecl56XbA,"Oxygen Bars, Event Planning & Services, Massag...",Las Vegas,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1,36.076518,-115.153343,The Body Connection,Southeast,89119,8,4.0,NV,"[Oxygen Bars, Event Planning & Services, Massa..."
107407,"1860 E Warner Rd, Ste 101","{'AcceptsInsurance': 'False', 'BikeParking': '...",bKoLvj4R8OUE2tFiqDvv_w,"Health & Medical, Beauty & Spas, Day Spas, Flo...",Tempe,"{'Monday': '16:0-22:0', 'Tuesday': '10:0-22:0'...",1,33.335594,-111.907285,True REST Float Spa,NaN,85284,133,4.5,AZ,"[Health & Medical, Beauty & Spas, Day Spas, Fl..."
118177,5875 S Rainbow Blvd 102,"{'AcceptsInsurance': 'False', 'BikeParking': '...",kV1VVT4ScYuZoXHAhWcDkg,"Float Spa, Massage Therapy, Halotherapy, Oxyge...",Las Vegas,"{'Monday': '16:0-22:0', 'Tuesday': '10:0-22:0'...",1,36.082052,-115.243265,True REST Float Spa,Spring Valley,89118,75,4.5,NV,"[Float Spa, Massage Therapy, Halotherapy, Oxyg..."


In [29]:
business_orig_dropna[business_orig_dropna["categories"].str.contains("Landmark") & ~business_orig_dropna["categories"].str.contains("Active Life")]

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,categories_clean
51,1 Queens Park Crescent E,NaN,wv9KN5x8L2qzKFq_6Hzf9g,"Public Services & Government, Landmarks & Hist...",Toronto,NaN,1,43.661964,-79.391259,Legislative Assembly of Ontario,Discovery District,M5S,3,4.5,ON,"[Public Services & Government, Landmarks & His..."
594,NaN,{'GoodForKids': 'True'},Bk2fENDaxjFLsx2_W3XKAw,"Public Services & Government, Landmarks & Hist...",Pittsburgh,NaN,1,40.441855,-80.012587,Point State Park Fountain,Downtown,15222,40,5.0,PA,"[Public Services & Government, Landmarks & His..."
1458,24139 Apache Trl,NaN,-dNbsz0MCWWP7tl1YS8pKA,"Landmarks & Historical Buildings, Public Servi...",Roosevelt,NaN,1,33.671444,-111.158846,Roosevelt Dam,NaN,85545,4,4.5,AZ,"[Landmarks & Historical Buildings, Public Serv..."
1601,NaN,NaN,jbJSFF0GcL3tfdyPASW1JA,"Public Services & Government, Landmarks & Hist...",Sainte-Anne-De-Bellevue,NaN,1,45.403962,-73.952575,Ville de Sainte-Anne-de-Bellevue,NaN,H9X 1M2,3,4.5,QC,"[Public Services & Government, Landmarks & His..."
3176,623 E 8th Ave,NaN,nMOGczQe3jcIfUNUHV8bwA,"Public Services & Government, Landmarks & Hist...",Homestead,"{'Monday': '10:0-14:0', 'Wednesday': '10:0-14:...",1,40.409180,-79.904372,Rivers of Steel National Heritage Area,Homestead,15120,12,4.0,PA,"[Public Services & Government, Landmarks & His..."
3361,600 Grant St,NaN,pbHZ6nD8Coz0FAnUva_Ggw,"Landmarks & Historical Buildings, Venues & Eve...",Pittsburgh,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1,40.441505,-79.995195,U.S. Steel Tower,Downtown,15219,5,4.5,PA,"[Landmarks & Historical Buildings, Venues & Ev..."
5218,6TH St,NaN,sKbso__tiJDw4tVqWpQHtA,"Landmarks & Historical Buildings, Public Servi...",Pittsburgh,NaN,1,40.444505,-80.002936,Roberto Clemente Bridge,Downtown,15272,22,4.5,PA,"[Landmarks & Historical Buildings, Public Serv..."
5383,4127 6 Street NE,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",9f1wph9lPCgGUSlprJQM0Q,"Nightlife, Bars, Landmarks & Historical Buildi...",Calgary,"{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'...",1,51.088938,-114.049141,Burwood Distillery,NaN,T2E 6V5,3,4.5,AB,"[Nightlife, Bars, Landmarks & Historical Build..."
6661,501 S College St,NaN,7S6WpJ-TQnFCXLiYsHKPcg,"Landmarks & Historical Buildings, Hotels & Tra...",Charlotte,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1,35.222351,-80.845683,Visit Charlotte,Uptown,28202,3,4.5,NC,"[Landmarks & Historical Buildings, Hotels & Tr..."
7469,9350 Highway 48,{'GoodForKids': 'True'},wrbKfIdC1Wq8pWB_qOipsw,"Arts & Entertainment, Public Services & Govern...",Markham,"{'Monday': '10:0-17:0', 'Tuesday': '10:0-17:0'...",1,43.856229,-79.337216,Markham Civic Centre,Brown's Corners,L3P 3J3,4,4.0,ON,"[Arts & Entertainment, Public Services & Gover..."


## Create one-hot-encode columns based on the top category terms.
1. Restaurants
2. Food
3. Shopping
4. Active Life
5. Arts & Entertainment
6. Nightlife
7. Hotels & Travel
8. Tours
9. Landmark

In [30]:
#for cat in category_list:
#    business_orig_dropna.drop(str('cat_' + cat), axis=1, inplace=True)

In [31]:
#business_orig_dropna.drop('cat_Others', axis=1, inplace=True)
category_list = ["Restaurants", "Food", "Shopping", "Active Life", "Arts & Entertainment", "Nightlife", "Hotels & Travel", "Tours", "Landmark"]

for cat in category_list:
    business_orig_dropna[str("cat_" + cat)] = np.where(business_orig_dropna['categories'].str.contains(cat), 1, 0)

business_orig_dropna.head(50)

C:\Users\mm4me\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,categories_clean,cat_Restaurants,cat_Food,cat_Shopping,cat_Active Life,cat_Arts & Entertainment,cat_Nightlife,cat_Hotels & Travel,cat_Tours,cat_Landmark
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,...,"[Tours, Breweries, Pizza, Restaurants, Food, H...",1,1,0,0,0,0,1,1,0
1,NaN,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,...,"[Chicken Wings, Burgers, Caterers, Street Vend...",1,1,0,0,0,0,0,0,0
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,...,"[Breakfast & Brunch, Restaurants, French, Sand...",1,0,0,0,0,0,0,0,0
3,211 W Monroe St,NaN,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,NaN,1,33.449999,-112.076979,Geico Insurance,...,"[Insurance, Financial Services]",0,0,0,0,0,0,0,0,0
4,2005 Alyth Place SE,{'BusinessAcceptsCreditCards': 'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,51.035591,-114.027366,Action Engine,...,"[Home & Garden, Nurseries & Gardening, Shoppin...",0,0,1,0,0,0,0,0,0
5,"20235 N Cave Creek Rd, Ste 1115","{'BikeParking': 'True', 'BusinessAcceptsCredit...",45bWSZtniwPRiqlivpS8Og,"Coffee & Tea, Food",Phoenix,"{'Monday': '5:30-20:0', 'Tuesday': '5:30-20:0'...",1,33.671375,-112.030017,The Coffee Bean & Tea Leaf,...,"[Coffee & Tea, Food]",0,1,0,0,0,0,0,0,0
6,631 Bloor St W,"{'BusinessParking': ""{'garage': False, 'street...",9A2quhZLyWk0akUetBd8hQ,"Food, Bakeries",Toronto,NaN,0,43.664378,-79.414424,Bnc Cake House,...,"[Food, Bakeries]",0,1,0,0,0,0,0,0,0
7,"3417 Derry Road E, Unit 103","{'Alcohol': 'none', 'BusinessAcceptsCreditCard...",6OuOZAok8ikONMS_T3EzXg,"Restaurants, Thai",Mississauga,NaN,1,43.712946,-79.632763,Thai One On,...,"[Restaurants, Thai]",1,0,0,0,0,0,0,0,0
8,1440 N. Dysart Ave,"{'Alcohol': 'none', 'Ambience': ""{'romantic': ...",8-NRKkPY1UiFXW20WXKiXg,"Mexican, Restaurants",Avondale,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,33.448106,-112.341302,Filiberto's Mexican Food,...,"[Mexican, Restaurants]",1,0,0,0,0,0,0,0,0
9,209 Oakland Ave,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",UTm5QZThPQlT35mkAcGOjg,"Flowers & Gifts, Gift Shops, Shopping",Pittsburgh,"{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ...",1,40.441421,-79.956457,Maggie & Stella's Gifts,...,"[Flowers & Gifts, Gift Shops, Shopping]",0,0,1,0,0,0,0,0,0


## Create 'cat_Others' column

In [32]:
# collection of category terms that are 0 in all 'category_list'
category_others = []
for i in business_orig_dropna.categories_clean[business_orig_dropna[(str("cat_" + x) for x in category_list)].apply(sum, axis=1) == 0]:
    category_others += i

# make the list unique
print(len(category_others))
category_others = list(sorted(set(category_others)))
print(len(category_others))

247722
632


In [33]:
sorted(category_others)

['& Probates',
 '3D Printing',
 'Accountants',
 'Acne Treatment',
 'Acupuncture',
 'Addiction Medicine',
 'Adoption Services',
 'Adult Education',
 'Advertising',
 'Aestheticians',
 'Air Duct Cleaning',
 'Aircraft Dealers',
 'Aircraft Repairs',
 'Allergists',
 'Alternative Medicine',
 'Anesthesiologists',
 'Animal Assisted Therapy',
 'Animal Physical Therapy',
 'Animal Shelters',
 'Apartment Agents',
 'Apartments',
 'Appliances & Repair',
 'Appraisal Services',
 'Aquarium Services',
 'Architects',
 'Art Classes',
 'Art Restoration',
 'Art Schools',
 'Art Space Rentals',
 'Artificial Turf',
 'Assisted Living Facilities',
 'Audio/Visual Equipment Rental',
 'Audiologist',
 'Auto Customization',
 'Auto Detailing',
 'Auto Electric Services',
 'Auto Glass Services',
 'Auto Insurance',
 'Auto Loan Providers',
 'Auto Parts & Supplies',
 'Auto Repair',
 'Auto Security',
 'Auto Upholstery',
 'Automotive',
 'Awnings',
 'Ayurveda',
 'Backflow Services',
 'Bail Bondsmen',
 'Balloon Services',
 'Ban

In [34]:
# take out 'Event Planning & Services', 'Caterers', 
category_others.remove('Event Planning & Services')
print(len(category_others))
category_others.remove('Party & Event Planning')
print(len(category_others))
category_others.remove('Venues & Event Spaces')
print(len(category_others))
category_others.remove('Caterers')
print(len(category_others))

# take out 'Local Flavor'
category_others.remove('Local Flavor')
print(len(category_others))

631
630
629
628
627


In [35]:
mask_other = business_orig_dropna.categories_clean.apply(lambda x: any(cat for cat in category_others if cat in x))
business_orig_dropna[mask_other]

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,categories_clean,cat_Restaurants,cat_Food,cat_Shopping,cat_Active Life,cat_Arts & Entertainment,cat_Nightlife,cat_Hotels & Travel,cat_Tours,cat_Landmark
3,211 W Monroe St,NaN,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,NaN,1,33.449999,-112.076979,Geico Insurance,...,"[Insurance, Financial Services]",0,0,0,0,0,0,0,0,0
4,2005 Alyth Place SE,{'BusinessAcceptsCreditCards': 'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,51.035591,-114.027366,Action Engine,...,"[Home & Garden, Nurseries & Gardening, Shoppin...",0,0,1,0,0,0,0,0,0
14,4404 14 Street NW,"{'BusinessParking': ""{'garage': False, 'street...",o_zDURWzgt5yM5HW_xGJsg,"Beauty & Spas, Barbers",Calgary,"{'Monday': '9:30-18:0', 'Tuesday': '9:30-18:0'...",1,51.091857,-114.094625,North Haven Barber Shop,...,"[Beauty & Spas, Barbers]",0,0,0,0,0,0,0,0,0
18,1549 N Rancho Dr,NaN,UwIpS9UKsPiKAv1fiEYhqg,"Financial Services, Check Cashing/Pay-day Loan...",Las Vegas,"{'Monday': '10:0-19:0', 'Tuesday': '10:0-18:0'...",1,36.188386,-115.186124,Nevada Title And Payday Loans,...,"[Financial Services, Check Cashing/Pay-day Loa...",0,0,0,0,0,0,0,0,0
20,NaN,{'BusinessAcceptsCreditCards': 'True'},YZCHr68c5aEVHz0bkq9K2g,"Home Services, Masonry/Concrete, Professional ...",Las Vegas,NaN,1,36.260816,-115.171130,Park Stone Pavers,...,"[Home Services, Masonry/Concrete, Professional...",0,0,0,0,0,0,0,0,0
26,NaN,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",guRyrKMBDkcHExGzwxSfvg,"Home Services, Home Automation, Security Systems",Phoenix,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,33.431665,-112.102959,"ADT Security Services, Inc.",...,"[Home Services, Home Automation, Security Syst...",0,0,0,0,0,0,0,0,0
28,1-905 Britannia Road West,NaN,65DAbjsiws2_T600pNpWFQ,"Optometrists, Shopping, Eyewear & Opticians, H...",Mississauga,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",1,43.609269,-79.699035,Laurier Optical,...,"[Optometrists, Shopping, Eyewear & Opticians, ...",0,0,1,0,0,0,0,0,0
31,3008 County Hwy T,"{'BusinessAcceptsCreditCards': 'True', 'Busine...",B_mz59xi-Q3zCIiXsnBM_A,"Shopping, Gardeners, Home Services, Nurseries ...",Sun Prairie,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1,43.122395,-89.239141,Schonheit Gardens,...,"[Shopping, Gardeners, Home Services, Nurseries...",0,0,1,0,0,0,0,0,0
34,"28325 N Tatum Blvd, Ste 5","{'AcceptsInsurance': 'False', 'BikeParking': '...",pbluh-ZZi_i8XlprfWB1nQ,"Health & Medical, Hair Salons, Skin Care, Hair...",Cave Creek,"{'Monday': '6:0-20:0', 'Tuesday': '6:0-20:0', ...",1,33.742488,-111.977420,Pamper Me Perfect Beauty & Wellness,...,"[Health & Medical, Hair Salons, Skin Care, Hai...",0,0,0,0,0,0,0,0,0
38,3319 S Mercy Rd,"{'AcceptsInsurance': 'True', 'ByAppointmentOnl...",iTZDPTc36guXhNhoK-jwmw,"Health & Medical, Physical Therapy, Retirement...",Gilbert,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,33.290596,-111.747814,Wellsprings of Gilbert,...,"[Health & Medical, Physical Therapy, Retiremen...",0,0,0,0,0,0,0,0,0


In [36]:
# create new column 'cat_Others'
#business_orig_dropna.drop('cat_Others', axis=1, inplace=True)
business_orig_dropna['cat_Others'] = np.where(business_orig_dropna.categories_clean.apply(lambda x: any(cat for cat in category_others if cat in x)), 1, 0)

C:\Users\mm4me\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
business_orig_dropna.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,cat_Restaurants,cat_Food,cat_Shopping,cat_Active Life,cat_Arts & Entertainment,cat_Nightlife,cat_Hotels & Travel,cat_Tours,cat_Landmark,cat_Others
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,...,1,1,0,0,0,0,1,1,0,0
1,NaN,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,...,1,1,0,0,0,0,0,0,0,0
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,...,1,0,0,0,0,0,0,0,0,0
3,211 W Monroe St,NaN,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,NaN,1,33.449999,-112.076979,Geico Insurance,...,0,0,0,0,0,0,0,0,0,1
4,2005 Alyth Place SE,{'BusinessAcceptsCreditCards': 'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,51.035591,-114.027366,Action Engine,...,0,0,1,0,0,0,0,0,0,1


In [38]:
business_orig_dropna[(business_orig_dropna['cat_Others'] == 1) & (business_orig_dropna['cat_Food'] == 1) & (business_orig_dropna['is_open'] == 1)]

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,cat_Restaurants,cat_Food,cat_Shopping,cat_Active Life,cat_Arts & Entertainment,cat_Nightlife,cat_Hotels & Travel,cat_Tours,cat_Landmark,cat_Others
272,4728 W Craig Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",pohAeCMsGb8p4dv6I_m-Kg,"Coffee & Tea, Automotive, Convenience Stores, ...",North Las Vegas,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,36.239341,-115.207056,7-Eleven,...,0,1,0,0,0,0,0,0,0,1
279,9920 West Flamingo Rd,"{'Ambience': ""{'romantic': False, 'intimate': ...",sOkQ2Y1-D8h18oN5NgxuLg,"Car Wash, Delis, Convenience Stores, Food, Res...",Las Vegas,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,36.115221,-115.309017,Haddies Car Wash and Convenience store,...,1,1,0,0,0,0,0,0,0,1
387,8402 W Thunderbird Rd,"{'Alcohol': 'none', 'BikeParking': 'True', 'Bu...",h8Pcv2krjGho2T-Mmgz7hA,"Health & Medical, Active Life, Vitamins & Supp...",Peoria,"{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'...",1,33.611479,-112.239838,Power Nutrition,...,0,1,1,1,0,0,0,0,0,1
854,8800 Euclid Chardon Rd,"{'BikeParking': 'True', 'BusinessAcceptsBitcoi...",xHfBxEQTSU43srGd4Wuo8w,"Food, Amusement Parks, Parks, Education, Museu...",Kirtland,"{'Tuesday': '9:0-17:0', 'Wednesday': '9:0-17:0...",1,41.581108,-81.331422,Lake Metroparks Farmpark,...,0,1,0,1,1,0,0,0,0,1
861,"2824 Prairie Lakes Dr, Ste 108","{'BusinessAcceptsCreditCards': 'True', 'Caters...",ZrKRPRwrBXayF6aZRh8lgg,"Beauty & Spas, Food, Nutritionists, Cosmetics ...",Sun Prairie,"{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'...",1,43.168084,-89.270254,Apple Wellness - Sun Prairie Health Store,...,0,1,1,0,0,0,0,0,0,1
1139,1141 Boulevard de Maisonneuve Ouest,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",xGw_X4Fwe5DvS-X-4NhUgg,"Financial Services, Food, Coffee & Tea, Banks ...",Montréal,"{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'...",1,45.500551,-73.575125,Tangerine - Café de Montréal,...,0,1,0,0,0,0,0,0,0,1
1143,3720 S Val Vista Dr,"{'Alcohol': 'beer_and_wine', 'BikeParking': 'T...",O32_y_U6dhjfoRLpUNGyJw,"Food, Automotive, Gas Stations, Convenience St...",Gilbert,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,33.282665,-111.756214,QuikTrip,...,0,1,0,0,0,0,0,0,0,1
1307,7747 N Tryon St,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",nOznuAwEDpD-zWxfmQobeA,"Automotive, Convenience Stores, Gas Stations, ...",Charlotte,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,35.294661,-80.755860,Circle K,...,0,1,0,0,0,0,0,0,0,1
1540,NaN,"{'BusinessAcceptsBitcoin': 'False', 'BusinessA...",ibBDBOJJbPF9qW36rK2OZw,"Caterers, Party & Event Planning, Bartenders, ...",Mesa,"{'Monday': '9:0-19:0', 'Tuesday': '9:0-19:0', ...",1,33.433516,-111.725694,Arizona Event Catering,...,0,1,0,0,0,0,0,0,0,1
1620,1520 N Scottsdale Rd,"{'BusinessAcceptsBitcoin': 'False', 'BusinessA...",mN88-7Qjf09azcWI-Dgyrg,"Food, Computers, Cupcakes, IT Services & Compu...",Tempe,"{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'...",1,33.446326,-111.926865,Technology Repair,...,0,1,1,0,0,0,0,0,0,1


## Take out unnecessary terms from 'categories_clean'

In [39]:
# take out main category terms
business_orig_dropna['categories_clean'] = business_orig_dropna['categories_clean'].apply(
                                            lambda x: [cat for cat in x if cat not in category_list])

business_orig_dropna['categories_clean']

C:\Users\mm4me\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0                                        [Breweries, Pizza]
1         [Chicken Wings, Burgers, Caterers, Street Vend...
2           [Breakfast & Brunch, French, Sandwiches, Cafes]
3                           [Insurance, Financial Services]
4         [Home & Garden, Nurseries & Gardening, Local S...
5                                            [Coffee & Tea]
6                                                [Bakeries]
7                                                    [Thai]
8                                                 [Mexican]
9                             [Flowers & Gifts, Gift Shops]
10                                               [Japanese]
11                                 [Cajun/Creole, Southern]
12        [Bars, Sports Bars, Dive Bars, Burgers, Sandwi...
13                      [Pakistani, Indian, Middle Eastern]
14                                 [Beauty & Spas, Barbers]
15                                      [Delis, Sandwiches]
16        [Bars, American (Traditional),

In [40]:
# take out category_other terms
business_orig_dropna['categories_clean'] = business_orig_dropna['categories_clean'].apply(
                                            lambda x: [cat for cat in x if cat not in category_others])

business_orig_dropna['categories_clean']

C:\Users\mm4me\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0                                        [Breweries, Pizza]
1         [Chicken Wings, Burgers, Caterers, Street Vend...
2           [Breakfast & Brunch, French, Sandwiches, Cafes]
3                                                        []
4                    [Home & Garden, Nurseries & Gardening]
5                                            [Coffee & Tea]
6                                                [Bakeries]
7                                                    [Thai]
8                                                 [Mexican]
9                             [Flowers & Gifts, Gift Shops]
10                                               [Japanese]
11                                 [Cajun/Creole, Southern]
12        [Bars, Sports Bars, Dive Bars, Burgers, Sandwi...
13                      [Pakistani, Indian, Middle Eastern]
14                                                       []
15                                      [Delis, Sandwiches]
16        [Bars, American (Traditional),

## Examine the sub category terms

In [41]:
# count the sub category terms
category_sub = []
for i in business_orig_dropna['categories_clean'][business_orig_dropna['is_open'] == 1]:
    category_sub += i

category_sub

len(set(category_sub))

661

In [42]:
Counter(category_sub).most_common()

[('Event Planning & Services', 8614),
 ('Bars', 7962),
 ('Fast Food', 6086),
 ('Fashion', 6046),
 ('Sandwiches', 5533),
 ('Coffee & Tea', 5525),
 ('Home & Garden', 5317),
 ('Pizza', 5179),
 ('American (Traditional)', 5024),
 ('Burgers', 4165),
 ('Breakfast & Brunch', 4031),
 ('Fitness & Instruction', 3773),
 ('Specialty Food', 3489),
 ('Italian', 3223),
 ('Mexican', 3216),
 ('Chinese', 3087),
 ('Grocery', 2901),
 ('American (New)', 2847),
 ('Bakeries', 2644),
 ('Hotels', 2266),
 ('Cafes', 2241),
 ("Women's Clothing", 2237),
 ('Desserts', 2228),
 ('Chicken Wings', 2167),
 ('Salad', 2001),
 ('Ice Cream & Frozen Yogurt', 1873),
 ('Caterers', 1852),
 ('Venues & Event Spaces', 1839),
 ('Gyms', 1832),
 ('Seafood', 1798),
 ('Cosmetics & Beauty Supply', 1791),
 ('Flowers & Gifts', 1759),
 ('Japanese', 1738),
 ('Home Decor', 1686),
 ('Sporting Goods', 1683),
 ('Accessories', 1648),
 ('Trainers', 1605),
 ('Convenience Stores', 1581),
 ('Pubs', 1565),
 ('Drugstores', 1547),
 ('Sushi Bars', 1526),

In [43]:
for i in range(1,11):
    print("There are {} sub-category terms that occur {} times.".format(len([k for k,v in Counter(category_sub).items() if v == i]), i))

There are 43 sub-category terms that occur 1 times.
There are 18 sub-category terms that occur 2 times.
There are 17 sub-category terms that occur 3 times.
There are 18 sub-category terms that occur 4 times.
There are 15 sub-category terms that occur 5 times.
There are 9 sub-category terms that occur 6 times.
There are 14 sub-category terms that occur 7 times.
There are 11 sub-category terms that occur 8 times.
There are 9 sub-category terms that occur 9 times.
There are 8 sub-category terms that occur 10 times.


In [44]:
len([k for k,v in Counter(category_sub).items() if v >= 10])

507

In [45]:
Counter(category_sub).most_common()

[('Event Planning & Services', 8614),
 ('Bars', 7962),
 ('Fast Food', 6086),
 ('Fashion', 6046),
 ('Sandwiches', 5533),
 ('Coffee & Tea', 5525),
 ('Home & Garden', 5317),
 ('Pizza', 5179),
 ('American (Traditional)', 5024),
 ('Burgers', 4165),
 ('Breakfast & Brunch', 4031),
 ('Fitness & Instruction', 3773),
 ('Specialty Food', 3489),
 ('Italian', 3223),
 ('Mexican', 3216),
 ('Chinese', 3087),
 ('Grocery', 2901),
 ('American (New)', 2847),
 ('Bakeries', 2644),
 ('Hotels', 2266),
 ('Cafes', 2241),
 ("Women's Clothing", 2237),
 ('Desserts', 2228),
 ('Chicken Wings', 2167),
 ('Salad', 2001),
 ('Ice Cream & Frozen Yogurt', 1873),
 ('Caterers', 1852),
 ('Venues & Event Spaces', 1839),
 ('Gyms', 1832),
 ('Seafood', 1798),
 ('Cosmetics & Beauty Supply', 1791),
 ('Flowers & Gifts', 1759),
 ('Japanese', 1738),
 ('Home Decor', 1686),
 ('Sporting Goods', 1683),
 ('Accessories', 1648),
 ('Trainers', 1605),
 ('Convenience Stores', 1581),
 ('Pubs', 1565),
 ('Drugstores', 1547),
 ('Sushi Bars', 1526),

In [46]:
# count the sub category terms for 'Restaurants' and 'Food'
category_sub_restaurantfood = []

for i in business_orig_dropna['categories_clean'][(business_orig_dropna['is_open'] == 1) & (business_orig_dropna['cat_Others'] == 0) & (business_orig_dropna['cat_Restaurants'] == 1) & (business_orig_dropna['cat_Food'] == 1)]:
    category_sub_restaurantfood += i

len(set(category_sub_restaurantfood))

337

In [47]:
len([k for k,v in Counter(category_sub_restaurantfood).items() if v >= 10])

179

In [48]:
Counter(category_sub_restaurantfood).most_common()

[('Fast Food', 6061),
 ('Sandwiches', 3025),
 ('Coffee & Tea', 2498),
 ('Burgers', 2415),
 ('Breakfast & Brunch', 1713),
 ('American (Traditional)', 1459),
 ('Cafes', 1422),
 ('Mexican', 1202),
 ('Specialty Food', 1170),
 ('Desserts', 1100),
 ('Bars', 1084),
 ('Bakeries', 1061),
 ('Pizza', 922),
 ('Delis', 868),
 ('Salad', 821),
 ('American (New)', 818),
 ('Chicken Wings', 809),
 ('Event Planning & Services', 735),
 ('Ice Cream & Frozen Yogurt', 668),
 ('Caterers', 638),
 ('Comfort Food', 636),
 ('Juice Bars & Smoothies', 599),
 ('Chinese', 596),
 ('Italian', 589),
 ('Beer', 574),
 ('Wine & Spirits', 574),
 ('Food Delivery Services', 477),
 ('Food Trucks', 469),
 ('Ethnic Food', 447),
 ('Seafood', 369),
 ('Grocery', 364),
 ('Canadian (New)', 353),
 ('Tex-Mex', 351),
 ('Chicken Shop', 347),
 ('Bagels', 324),
 ('Asian Fusion', 311),
 ('Barbeque', 304),
 ('Gluten-Free', 281),
 ('Vegetarian', 277),
 ('Hot Dogs', 277),
 ('Imported Food', 263),
 ('Vegan', 262),
 ('Soup', 261),
 ('Mediterrane

In [49]:
# count the sub category terms for 'Restaurants' and 'Food' AND 'Las Vegas'
category_sub_restaurantfood_lv = []

for i in business_orig_dropna['categories_clean'][(business_orig_dropna['is_open'] == 1) & (business_orig_dropna['cat_Restaurants'] == 1) | (business_orig_dropna['cat_Food'] == 1) & (business_orig_dropna['city'] == 'Las Vegas')]:
    category_sub_restaurantfood_lv += i

len(set(category_sub_restaurantfood_lv))

475

In [50]:
len([k for k,v in Counter(category_sub_restaurantfood_lv).items() if v >= 50])

163

In [51]:
# count the sub category terms for 'Shopping'
category_sub_shopping = []

for i in business_orig_dropna['categories_clean'][(business_orig_dropna['is_open'] == 1) & (business_orig_dropna['cat_Shopping'] == 1) & (business_orig_dropna['city'] == 'Las Vegas')]:
    category_sub_shopping += i

len(set(category_sub_shopping))

317

In [52]:
len([k for k,v in Counter(category_sub_shopping).items() if v >= 30])

73

In [53]:
Counter(category_sub_shopping).most_common()

[('Fashion', 1135),
 ('Home & Garden', 669),
 ("Women's Clothing", 447),
 ('Cosmetics & Beauty Supply', 364),
 ('Accessories', 363),
 ("Men's Clothing", 323),
 ('Shoe Stores', 272),
 ('Jewelry', 252),
 ('Home Decor', 211),
 ('Electronics', 207),
 ('Drugstores', 204),
 ('Flowers & Gifts', 202),
 ('Event Planning & Services', 198),
 ('Sporting Goods', 190),
 ('Department Stores', 185),
 ('Furniture Stores', 183),
 ('Mobile Phones', 172),
 ('Tobacco Shops', 159),
 ('Arts & Crafts', 151),
 ('Discount Store', 145),
 ('Eyewear & Opticians', 122),
 ('Kitchen & Bath', 114),
 ('Art Galleries', 110),
 ('Vape Shops', 106),
 ('Grocery', 103),
 ('Books', 102),
 ('Mags', 102),
 ('Music & Video', 102),
 ('Florists', 97),
 ('Sports Wear', 96),
 ('Mobile Phone Accessories', 94),
 ('Gift Shops', 92),
 ('Watches', 92),
 ('Outlet Stores', 89),
 ('Photography Stores & Services', 83),
 ('Specialty Food', 78),
 ('Appliances', 77),
 ('Bridal', 76),
 ('Mattresses', 75),
 ('Convenience Stores', 74),
 ('Hobby Sh

In [54]:
# count the sub category terms for 'Active Life'
category_sub_active = []

for i in business_orig_dropna['categories_clean'][(business_orig_dropna['is_open'] == 1) & (business_orig_dropna['cat_Active Life'] == 1) & (business_orig_dropna['cat_Tours'] == 1) & (business_orig_dropna['city'] == 'Las Vegas')]:
    category_sub_active += i

len(set(category_sub_active))

62

In [55]:
len([k for k,v in Counter(category_sub_active).items() if v >= 2])

32

In [56]:
Counter(category_sub_active).most_common()

[('ATV Rentals/Tours', 17),
 ('Bike Rentals', 11),
 ('Travel Services', 10),
 ('Event Planning & Services', 10),
 ('Motorcycle Rental', 7),
 ('Hiking', 7),
 ('Sporting Goods', 7),
 ('Walking Tours', 6),
 ('Amusement Parks', 6),
 ('Hot Air Balloons', 5),
 ('Car Rental', 4),
 ('Bikes', 4),
 ('Skydiving', 4),
 ('Venues & Event Spaces', 4),
 ('Mountain Biking', 4),
 ('Scooter Rentals', 3),
 ('Outdoor Gear', 3),
 ('Historical Tours', 3),
 ('Fitness & Instruction', 3),
 ('Landmarks & Historical Buildings', 3),
 ('Local Flavor', 3),
 ('Scavenger Hunts', 3),
 ('Party & Event Planning', 3),
 ('Aerial Tours', 3),
 ('Parks', 3),
 ('Scooter Tours', 2),
 ('Climbing', 2),
 ('Bars', 2),
 ('Rafting/Kayaking', 2),
 ('Food Tours', 2),
 ('Bike tours', 2),
 ('Recreation Centers', 2),
 ('Sailing', 1),
 ('Golf', 1),
 ('Beer', 1),
 ('Wine & Spirits', 1),
 ('Golf Equipment', 1),
 ('Challenge Courses', 1),
 ('Diving', 1),
 ('Dive Shops', 1),
 ('Scuba Diving', 1),
 ('Snorkeling', 1),
 ('Ski Resorts', 1),
 ('Tra

In [57]:
# count the sub category terms for 'Arts & Entertainment'
category_sub_arts = []

for i in business_orig_dropna['categories_clean'][(business_orig_dropna['is_open'] == 1) & (business_orig_dropna['cat_Arts & Entertainment'] == 1) & (business_orig_dropna['city'] == 'Las Vegas')]:
    category_sub_arts += i

len(set(category_sub_arts))

253

In [58]:
len([k for k,v in Counter(category_sub_arts).items() if v >= 10])

51

In [59]:
Counter(category_sub_arts).most_common()

[('Performing Arts', 307),
 ('Event Planning & Services', 254),
 ('Casinos', 207),
 ('Music Venues', 149),
 ('Bars', 119),
 ('Art Galleries', 110),
 ('Hotels', 96),
 ('Festivals', 81),
 ('Party & Event Planning', 60),
 ('Venues & Event Spaces', 56),
 ('Local Flavor', 48),
 ('Arcades', 45),
 ('Supernatural Readings', 44),
 ('Adult Entertainment', 42),
 ('Fitness & Instruction', 41),
 ('Dance Clubs', 41),
 ('American (Traditional)', 36),
 ('Lounges', 35),
 ('Museums', 35),
 ('Ticket Sales', 35),
 ('Resorts', 35),
 ('Cinema', 33),
 ('Comedy Clubs', 28),
 ('Stadiums & Arenas', 25),
 ('Social Clubs', 25),
 ('Psychics', 24),
 ('Dance Studios', 24),
 ('Amusement Parks', 23),
 ('Arts & Crafts', 21),
 ('American (New)', 21),
 ('Cocktail Bars', 19),
 ('Books', 18),
 ('Mags', 18),
 ('Music & Video', 18),
 ('Pubs', 17),
 ('Sports Bars', 16),
 ('Home & Garden', 15),
 ('Travel Services', 15),
 ('Fashion', 14),
 ('Cultural Center', 12),
 ('Race Tracks', 12),
 ('Spiritual Shop', 11),
 ('Cabaret', 11),

In [60]:
# count the sub category terms for 'Nightlife'
category_sub_night = []

for i in business_orig_dropna['categories_clean'][(business_orig_dropna['is_open'] == 1) & (business_orig_dropna['cat_Nightlife'] == 1) & (business_orig_dropna['city'] == 'Las Vegas')]:
    category_sub_night += i

len(set(category_sub_night))

253

In [61]:
len([k for k,v in Counter(category_sub_night).items() if v >= 10])

65

In [62]:
Counter(category_sub_night).most_common()

[('Bars', 1043),
 ('American (Traditional)', 243),
 ('Lounges', 231),
 ('Sports Bars', 214),
 ('American (New)', 174),
 ('Event Planning & Services', 164),
 ('Cocktail Bars', 149),
 ('Music Venues', 149),
 ('Dance Clubs', 144),
 ('Pubs', 142),
 ('Adult Entertainment', 114),
 ('Breakfast & Brunch', 81),
 ('Performing Arts', 79),
 ('Beer', 75),
 ('Wine & Spirits', 75),
 ('Party & Event Planning', 72),
 ('Pizza', 62),
 ('Wine Bars', 61),
 ('Burgers', 59),
 ('Casinos', 55),
 ('Steakhouses', 54),
 ('Dive Bars', 52),
 ('Mexican', 51),
 ('Seafood', 50),
 ('Karaoke', 50),
 ('Venues & Event Spaces', 49),
 ('Italian', 47),
 ('Comedy Clubs', 39),
 ('Hookah Bars', 37),
 ('Beer Bar', 35),
 ('Asian Fusion', 31),
 ('Hotels', 28),
 ('Sandwiches', 28),
 ('Swimming Pools', 28),
 ('Salad', 27),
 ('Gastropubs', 27),
 ('Desserts', 26),
 ('Transportation', 24),
 ('Coffee & Tea', 24),
 ('Travel Services', 23),
 ('Limos', 23),
 ('Pool Halls', 23),
 ('Chicken Wings', 22),
 ('Tobacco Shops', 20),
 ('Breweries',

In [63]:
# count the sub category terms for 'Hotels & Travel'
category_sub_hotel = []

for i in business_orig_dropna['categories_clean'][(business_orig_dropna['is_open'] == 1) & (business_orig_dropna['cat_Hotels & Travel'] == 1) & (business_orig_dropna['city'] == 'Las Vegas')]:
    category_sub_hotel += i

len(set(category_sub_hotel))

170

In [64]:
len([k for k,v in Counter(category_sub_hotel).items() if v >= 10])

34

In [65]:
Counter(category_sub_hotel).most_common()

[('Event Planning & Services', 408),
 ('Hotels', 309),
 ('Transportation', 181),
 ('Car Rental', 143),
 ('Travel Services', 103),
 ('Casinos', 97),
 ('Limos', 85),
 ('Resorts', 69),
 ('Party & Event Planning', 56),
 ('Airport Shuttles', 51),
 ('Venues & Event Spaces', 49),
 ('Bars', 40),
 ('Taxis', 38),
 ('Dance Clubs', 37),
 ('Airports', 27),
 ('Vacation Rentals', 27),
 ('Bus Tours', 24),
 ('Airlines', 24),
 ('Walking Tours', 22),
 ('Motorcycle Rental', 18),
 ('Public Transportation', 18),
 ('Buses', 17),
 ('RV Rental', 16),
 ('RV Parks', 16),
 ('ATV Rentals/Tours', 15),
 ('Adult Entertainment', 15),
 ('Bike Rentals', 13),
 ('Vacation Rental Agents', 13),
 ('Club Crawl', 13),
 ('Campgrounds', 12),
 ('Historical Tours', 11),
 ('Hiking', 10),
 ('Music Venues', 10),
 ('Travel Agents', 10),
 ('Lounges', 9),
 ('Sporting Goods', 8),
 ('American (Traditional)', 8),
 ('Hostels', 8),
 ('Beer', 7),
 ('Wine & Spirits', 7),
 ('Local Flavor', 7),
 ('Performing Arts', 7),
 ('Swimming Pools', 7),
 (

In [66]:
# count the sub category terms for 'Tours'
category_sub_tours = []

for i in business_orig_dropna['categories_clean'][(business_orig_dropna['is_open'] == 1) & (business_orig_dropna['cat_Tours'] == 1) & (business_orig_dropna['city'] == 'Las Vegas')]:
    category_sub_tours += i

len(set(category_sub_tours))

98

In [67]:
len([k for k,v in Counter(category_sub_tours).items() if v >= 5])

29

In [68]:
Counter(category_sub_tours).most_common()

[('Event Planning & Services', 47),
 ('Travel Services', 33),
 ('Transportation', 30),
 ('Bus Tours', 24),
 ('Party & Event Planning', 23),
 ('Walking Tours', 22),
 ('Limos', 19),
 ('ATV Rentals/Tours', 17),
 ('Bars', 14),
 ('Dance Clubs', 14),
 ('Airport Shuttles', 14),
 ('Club Crawl', 12),
 ('Motorcycle Rental', 11),
 ('Bike Rentals', 11),
 ('Historical Tours', 11),
 ('Car Rental', 9),
 ('Hiking', 7),
 ('Sporting Goods', 7),
 ('Venues & Event Spaces', 7),
 ('Adult Entertainment', 7),
 ('Food Tours', 7),
 ('Aerial Tours', 6),
 ('Cannabis Tours', 6),
 ('Buses', 6),
 ('Amusement Parks', 6),
 ('Landmarks & Historical Buildings', 5),
 ('Local Flavor', 5),
 ('Museums', 5),
 ('Hot Air Balloons', 5),
 ('Town Car Service', 4),
 ('Taxis', 4),
 ('Cannabis Dispensaries', 4),
 ('Bikes', 4),
 ('Skydiving', 4),
 ('Mountain Biking', 4),
 ('Hotels', 4),
 ('Scooter Rentals', 3),
 ('Outdoor Gear', 3),
 ('Fitness & Instruction', 3),
 ('Travel Agents', 3),
 ('Scavenger Hunts', 3),
 ('Parks', 3),
 ('Art T

In [69]:
# count the sub category terms for 'Landmark'
category_sub_landmark = []

for i in business_orig_dropna['categories_clean'][(business_orig_dropna['is_open'] == 1) & (business_orig_dropna['cat_Landmark'] == 1) & (business_orig_dropna['city'] == 'Las Vegas')]:
    category_sub_landmark += i

len(set(category_sub_landmark))

18

In [70]:
len([k for k,v in Counter(category_sub_landmark).items() if v >= 2])

4

In [71]:
Counter(category_sub_landmark).most_common()

[('Landmarks & Historical Buildings', 21),
 ('Local Flavor', 3),
 ('Parks', 2),
 ('Museums', 2),
 ('Amusement Parks', 1),
 ('Bus Tours', 1),
 ('Travel Services', 1),
 ('Transportation', 1),
 ('Event Planning & Services', 1),
 ('Venues & Event Spaces', 1),
 ('Adult Entertainment', 1),
 ('Art Galleries', 1),
 ('Historical Tours', 1),
 ('Walking Tours', 1),
 ('Gun/Rifle Ranges', 1),
 ('Burgers', 1),
 ('Performing Arts', 1),
 ('Street Art', 1)]

In [72]:
# save the list of sub-category terms using pickle
with open("sub_category/sub_cat.pkl", "wb") as f:
    pickle.dump(category_sub, f)

In [73]:
# 163
with open("sub_category/restaurantfood.pkl", "wb") as f:
    pickle.dump([k for k,v in Counter(category_sub_restaurantfood_lv).items() if v >= 50], f)

In [74]:
# 73 
with open("sub_category/shopping.pkl", "wb") as f:
    pickle.dump([k for k,v in Counter(category_sub_shopping).items() if v >= 30], f)

# 32
with open("sub_category/active.pkl", "wb") as f:
    pickle.dump([k for k,v in Counter(category_sub_active).items() if v >= 2], f)
    
# 51
with open("sub_category/arts.pkl", "wb") as f:
    pickle.dump([k for k,v in Counter(category_sub_arts).items() if v >= 10], f)
    
# 65
with open("sub_category/night.pkl", "wb") as f:
    pickle.dump([k for k,v in Counter(category_sub_night).items() if v >= 10], f)
    
# 34
with open("sub_category/hotel.pkl", "wb") as f:
    pickle.dump([k for k,v in Counter(category_sub_hotel).items() if v >= 10], f)
    
# 29
with open("sub_category/tours.pkl", "wb") as f:
    pickle.dump([k for k,v in Counter(category_sub_tours).items() if v >= 5], f)
    
# 4
with open("sub_category/landmark.pkl", "wb") as f:
    pickle.dump([k for k,v in Counter(category_sub_landmark).items() if v >= 2], f)

## Merge new columns to the original data frame

In [75]:
print(business.shape)
print(business_orig.shape)
print(business_orig_dropna.shape)

(188593, 72)
(188593, 16)
(188052, 26)


In [76]:
# merge newly created category columns to the business matrix
#business.drop(business.columns[0], axis=1, inplace=True)
business_full = business.merge(business_orig_dropna, how="left", on="business_id", suffixes=('','_y'))
print(business_full.shape)

(188593, 97)


In [77]:
#business_full.drop(business_full.columns[0], axis=1, inplace=True)
business_full.drop(list(business_full.filter(regex='_y')), axis=1, inplace=True)
business_full.head()

,business_id,total_counts,45_counts,text,text_friendly,text_reasonable,text_local,text_comfortable,text_easy,text_quick,...,cat_Restaurants,cat_Food,cat_Shopping,cat_Active Life,cat_Arts & Entertainment,cat_Nightlife,cat_Hotels & Travel,cat_Tours,cat_Landmark,cat_Others
0,--1UhMGODdWsrMastO9DZw,24,19.0,Came here for lunch last week and was pleasant...,0.291667,0.041667,0.000000,0.000000,0.083333,0.125000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,--6MefnULPED_I942VcFNA,39,17.0,Johns Chinese BBQ ruined Char Siu Pork for me....,0.051282,0.000000,0.025641,0.000000,0.000000,0.051282,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,--7zmmkVg-IMGaXbuVd0SQ,54,39.0,"Great beer, great place, excellent service fro...",0.166667,0.000000,0.129630,0.055556,0.111111,0.018519,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,--8LPVSo5i0Oo61X01sV9A,4,2.0,Dr. LaCognata is great. He is a great listene...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,--9QQLMTbFzLJ_oT-ON3Xw,11,7.0,I've been going to this particular location fo...,0.000000,0.000000,0.000000,0.090909,0.090909,0.181818,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [78]:
# drop duplicate columns
business_full.drop(['total_counts', '45_counts'], axis=1, inplace=True)

business_full.shape

(188593, 81)

#### Need to replace the current 'text' column as it only contains 4 or 5 star review text

In [79]:
review = pd.read_csv("review.csv")
review.shape

(5996996, 9)

In [80]:
review.isnull().sum()

business_id    0
cool           0
date           0
funny          0
review_id      0
stars          0
text           1
useful         0
user_id        0
dtype: int64

In [81]:
# there is one review without any text or stars
review[review.text.isnull()]

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
4744944,FgNgBLayRFm6H6Qr66ecbQ,0,2018-06-17,0,QW01qOsaqlxMKoMazOw1Bg,0,NaN,0,wrqh88xVEE1U5d_5TjzN4Q


In [82]:
# drop the row
review.drop(review[review.text.isnull()].index, inplace=True)

In [83]:
df_text = review.groupby('business_id')['text'].apply(' '.join).reset_index()
df_text.head()

,business_id,text
0,--1UhMGODdWsrMastO9DZw,Came here for lunch last week and was pleasant...
1,--6MefnULPED_I942VcFNA,The incredibly rude woman behind the cashier w...
2,--7zmmkVg-IMGaXbuVd0SQ,"Great beer, great place, excellent service fro..."
3,--8LPVSo5i0Oo61X01sV9A,Dr. LaCognata is great. He is a great listene...
4,--9QQLMTbFzLJ_oT-ON3Xw,I've been going to this particular location fo...


In [84]:
print(df_text.shape)

(188593, 2)


In [85]:
# merge the new text column
business_full = business_full.merge(df_text, how='left', on='business_id', suffixes=('', '_y'))
business_full.head()

,business_id,text,text_friendly,text_reasonable,text_local,text_comfortable,text_easy,text_quick,1star_counts,2star_counts,...,cat_Food,cat_Shopping,cat_Active Life,cat_Arts & Entertainment,cat_Nightlife,cat_Hotels & Travel,cat_Tours,cat_Landmark,cat_Others,text_y
0,--1UhMGODdWsrMastO9DZw,Came here for lunch last week and was pleasant...,0.291667,0.041667,0.000000,0.000000,0.083333,0.125000,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Came here for lunch last week and was pleasant...
1,--6MefnULPED_I942VcFNA,Johns Chinese BBQ ruined Char Siu Pork for me....,0.051282,0.000000,0.025641,0.000000,0.000000,0.051282,6.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,The incredibly rude woman behind the cashier w...
2,--7zmmkVg-IMGaXbuVd0SQ,"Great beer, great place, excellent service fro...",0.166667,0.000000,0.129630,0.055556,0.111111,0.018519,5.0,4.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Great beer, great place, excellent service fro..."
3,--8LPVSo5i0Oo61X01sV9A,Dr. LaCognata is great. He is a great listene...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Dr. LaCognata is great. He is a great listene...
4,--9QQLMTbFzLJ_oT-ON3Xw,I've been going to this particular location fo...,0.000000,0.000000,0.000000,0.090909,0.090909,0.181818,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,I've been going to this particular location fo...


In [86]:
# drop the origial text column and rename
business_full.drop(['text'], axis=1, inplace=True)

In [87]:
business_full.rename({'text_y': 'text'}, axis=1, inplace=True)
business_full.head()

,business_id,text_friendly,text_reasonable,text_local,text_comfortable,text_easy,text_quick,1star_counts,2star_counts,3star_counts,...,cat_Food,cat_Shopping,cat_Active Life,cat_Arts & Entertainment,cat_Nightlife,cat_Hotels & Travel,cat_Tours,cat_Landmark,cat_Others,text
0,--1UhMGODdWsrMastO9DZw,0.291667,0.041667,0.000000,0.000000,0.083333,0.125000,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Came here for lunch last week and was pleasant...
1,--6MefnULPED_I942VcFNA,0.051282,0.000000,0.025641,0.000000,0.000000,0.051282,6.0,6.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,The incredibly rude woman behind the cashier w...
2,--7zmmkVg-IMGaXbuVd0SQ,0.166667,0.000000,0.129630,0.055556,0.111111,0.018519,5.0,4.0,6.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Great beer, great place, excellent service fro..."
3,--8LPVSo5i0Oo61X01sV9A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Dr. LaCognata is great. He is a great listene...
4,--9QQLMTbFzLJ_oT-ON3Xw,0.000000,0.000000,0.000000,0.090909,0.090909,0.181818,3.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,I've been going to this particular location fo...


In [88]:
# reorder columns
col_order = ['business_id', 'name', 'city', 'is_open', 'stars', 'price',           
             'review_count', '1star_counts', '2star_counts', '3star_counts', '4star_counts', '5star_counts',
             'text', 'text_friendly', 'text_reasonable', 'text_local', 'text_comfortable', 'text_easy', 'text_quick',
             'categories', 'categories_clean', 'cat_Restaurants', 'cat_Food', 'cat_Shopping', 'cat_Active Life',
             'cat_Arts & Entertainment', 'cat_Nightlife', 'cat_Hotels & Travel', 'cat_Tours', 'cat_Landmark', 'cat_Others',
             'state','address', 'neighborhood', 'postal_code', 'latitude', 'longitude',
             'hours', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday',
             'attributes', 'agesallowed', 'alcohol', 'ambience', 'byob', 'byobcorkage', 'bestnights', 'bikeparking',
             'businessacceptsbitcoin', 'businessacceptscreditcards', 'businessparking', 'byappointmentonly', 'coatcheck',
             'corkage', 'dietaryrestrictions', 'dogsallowed', 'drivethru', 'goodfordancing', 'goodforkids', 'goodformeal',
             'happyhour', 'hastv', 'music', 'noiselevel', 'open24hours', 'outdoorseating', 'restaurantsattire',
             'restaurantscounterservice', 'restaurantsdelivery', 'restaurantsgoodforgroups', 'restaurantsreservations',
             'restaurantstableservice', 'restaurantstakeout', 'smoking', 'wheelchairaccessible', 'wifi'    
            ]

len(col_order)

81

In [89]:
business_full = business_full[col_order]
business_full.head()

,business_id,name,city,is_open,stars,price,review_count,1star_counts,2star_counts,3star_counts,...,restaurantsattire,restaurantscounterservice,restaurantsdelivery,restaurantsgoodforgroups,restaurantsreservations,restaurantstableservice,restaurantstakeout,smoking,wheelchairaccessible,wifi
0,--1UhMGODdWsrMastO9DZw,The Spicy Amigos,Calgary,1,4.0,2.0,24,2.0,2.0,1.0,...,casual,NaN,False,True,NaN,False,True,NaN,NaN,NaN
1,--6MefnULPED_I942VcFNA,John's Chinese BBQ Restaurant,Richmond Hill,1,3.0,2.0,39,6.0,6.0,10.0,...,casual,NaN,False,True,True,True,True,NaN,True,no
2,--7zmmkVg-IMGaXbuVd0SQ,Primal Brewery,Huntersville,1,4.0,1.0,54,5.0,4.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,True,NaN
3,--8LPVSo5i0Oo61X01sV9A,Valley Bone and Joint Specialists,Gilbert,1,3.5,NaN,4,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,--9QQLMTbFzLJ_oT-ON3Xw,Great Clips,Tempe,1,3.5,1.0,11,3.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
business_full.set_index("business_id", inplace=True)

In [91]:
# save file
business_full.to_csv("business_full.csv")

### Filter the dataset to Open & Top 5 Cities

In [92]:
# load file
business_full = pd.read_csv('business_full.csv', index_col='business_id')
business_full.shape

C:\Users\mm4me\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (49,50,58,69,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(188593, 80)

In [92]:
# save only the categories of interest
#business_partial = business_full[business_full[(str("cat_" + x) for x in category_list)].apply(sum, axis=1) >= 1]
business_partial = business_full[business_full["cat_Others"] == 0]
print(business_partial.shape)

# subset to businesses that are open
business_partial = business_partial[business_partial["is_open"] == int(1)]
print(business_partial.shape)

# subset businesses in top 5 cities
business_partial = business_partial[business_partial["city"].isin(['Las Vegas', 'Phoenix', 'Scottsdale', 'Charlotte', 'Toronto'])]
business_partial.head()

(102540, 80)
(77997, 80)


,name,city,is_open,stars,price,review_count,1star_counts,2star_counts,3star_counts,4star_counts,...,restaurantsattire,restaurantscounterservice,restaurantsdelivery,restaurantsgoodforgroups,restaurantsreservations,restaurantstableservice,restaurantstakeout,smoking,wheelchairaccessible,wifi
business_id,,,,,,,,,,,,,,,,,,,,,
--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,Las Vegas,1,4.0,4.0,1546,51.0,137.0,176.0,420.0,...,dressy,NaN,False,True,True,True,False,NaN,True,no
--DaPTJW3-tB1vP-PfdTEg,Sunnyside Grill,Toronto,1,3.5,2.0,45,4.0,3.0,9.0,16.0,...,casual,NaN,False,True,False,True,True,NaN,NaN,free
--DdmeR16TRb3LsjG0ejrQ,World Food Championships,Las Vegas,1,3.0,NaN,5,0.0,2.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--KCl2FvVQpvjzmZSPyviA,Hungry Howie's Pizza,Charlotte,1,3.0,1.0,13,3.0,2.0,3.0,1.0,...,casual,NaN,True,True,False,False,True,NaN,True,no
--cZ6Hhc9F7VkKXxHMVZSQ,Pio Pio,Charlotte,1,4.0,2.0,333,22.0,22.0,43.0,111.0,...,casual,NaN,True,True,True,True,True,NaN,True,no


In [93]:
print(business_full.shape)
print(business_partial.shape)

(188593, 80)
(30297, 80)


In [94]:
# group list of columns
features_loc = ['state', 'address', 'neighborhood', 'postal_code', 'latitude', 'longitude']

features_op = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']

features_attr = ['attributes', 'agesallowed', 'alcohol', 'ambience', 'byob', 'byobcorkage', 'bestnights', 'bikeparking',
                 'businessacceptsbitcoin', 'businessacceptscreditcards', 'businessparking', 'byappointmentonly',
                 'coatcheck', 'corkage', 'dietaryrestrictions', 'dogsallowed', 'drivethru', 'goodfordancing',
                 'goodforkids', 'goodformeal', 'happyhour', 'hastv', 'music', 'noiselevel', 'open24hours',
                 'outdoorseating', 'restaurantsattire', 'restaurantscounterservice', 'restaurantsdelivery',
                 'restaurantsgoodforgroups', 'restaurantsreservations', 'restaurantstableservice', 'restaurantstakeout',
                 'smoking', 'wheelchairaccessible', 'wifi', 'hours']

In [95]:
business_partial[features_op].isnull().sum()/business_partial.shape[0]

monday       0.279170
tuesday      0.250025
wednesday    0.242532
thursday     0.239034
friday       0.236822
saturday     0.259696
sunday       0.339440
dtype: float64

In [96]:
# fill with null
business_partial[features_op] = business_partial[features_op].fillna(0)
business_partial[features_op].head()

,monday,tuesday,wednesday,thursday,friday,saturday,sunday
business_id,,,,,,,
--9e1ONYQuAa-CB_Rrw7Tw,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0
--DaPTJW3-tB1vP-PfdTEg,7:0-15:0,7:0-15:0,7:0-15:0,7:0-15:0,7:0-16:0,7:0-16:0,7:0-16:0
--DdmeR16TRb3LsjG0ejrQ,0,0,0,0,10:0-22:0,10:0-22:0,10:0-19:0
--KCl2FvVQpvjzmZSPyviA,10:30-0:0,10:30-0:0,10:30-0:0,10:30-0:0,10:30-1:0,10:30-1:0,10:30-0:0
--cZ6Hhc9F7VkKXxHMVZSQ,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,12:0-21:0


In [97]:
# examining other attribute column reveals that majority are missing
business_partial[features_attr].isnull().sum()/business_partial.shape[0]

attributes                    0.058059
agesallowed                   0.995511
alcohol                       0.561046
ambience                      1.000000
byob                          0.990296
byobcorkage                   0.985015
bestnights                    1.000000
bikeparking                   0.272073
businessacceptsbitcoin        0.936528
businessacceptscreditcards    0.204311
businessparking               1.000000
byappointmentonly             0.933228
coatcheck                     0.910948
corkage                       0.991418
dietaryrestrictions           1.000000
dogsallowed                   0.868040
drivethru                     0.925636
goodfordancing                0.901244
goodforkids                   0.466680
goodformeal                   1.000000
happyhour                     0.900980
hastv                         0.555104
music                         1.000000
noiselevel                    0.599003
open24hours                   0.995478
outdoorseating           

In [98]:
# drop attributes
business_partial.drop([col for col in features_attr], axis=1, inplace=True)

#### Create new column las vegas

In [99]:
# create column denoting Las Vegas or other
business_partial['las_vegas'] = np.where(business_partial['city'] == 'Las Vegas', 1, 0)
business_partial.head()

,name,city,is_open,stars,price,review_count,1star_counts,2star_counts,3star_counts,4star_counts,...,latitude,longitude,monday,tuesday,wednesday,thursday,friday,saturday,sunday,las_vegas
business_id,,,,,,,,,,,,,,,,,,,,,
--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,Las Vegas,1,4.0,4.0,1546,51.0,137.0,176.0,420.0,...,36.123183,-115.169190,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,1
--DaPTJW3-tB1vP-PfdTEg,Sunnyside Grill,Toronto,1,3.5,2.0,45,4.0,3.0,9.0,16.0,...,43.677807,-79.444674,7:0-15:0,7:0-15:0,7:0-15:0,7:0-15:0,7:0-16:0,7:0-16:0,7:0-16:0,0
--DdmeR16TRb3LsjG0ejrQ,World Food Championships,Las Vegas,1,3.0,NaN,5,0.0,2.0,0.0,3.0,...,36.114277,-115.170975,0,0,0,0,10:0-22:0,10:0-22:0,10:0-19:0,1
--KCl2FvVQpvjzmZSPyviA,Hungry Howie's Pizza,Charlotte,1,3.0,1.0,13,3.0,2.0,3.0,1.0,...,35.333478,-80.796171,10:30-0:0,10:30-0:0,10:30-0:0,10:30-0:0,10:30-1:0,10:30-1:0,10:30-0:0,0
--cZ6Hhc9F7VkKXxHMVZSQ,Pio Pio,Charlotte,1,4.0,2.0,333,22.0,22.0,43.0,111.0,...,35.199853,-80.844820,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,12:0-21:0,0


#### Create new column review word count

In [100]:
business_partial['word_count'] = business_partial.text.str.split().str.len()/business_partial.review_count

In [101]:
# drop is_open
business_partial.drop(columns='is_open', inplace=True)

In [102]:
business_partial.shape

(30297, 44)

In [103]:
# save file
business_partial.to_csv("business_partial.csv")

### Filter the data to Restaurants & Food only

In [99]:
# load file
business_partial = pd.read_csv('business_partial.csv', index_col='business_id')

In [104]:
# create Restaurant/Food data frame with sub-category columns
business_r_and_f = business_partial[(business_partial["cat_Restaurants"] == 1) | (business_partial["cat_Food"] == 1)]
business_r_and_f.shape

(18504, 44)

In [105]:
# compare with a different filter method
mask_non = (business_partial["cat_Shopping"] == 0) & (business_partial["cat_Active Life"] == 0) & (business_partial["cat_Arts & Entertainment"] == 0) & (business_partial["cat_Nightlife"] == 0) & (business_partial["cat_Tours"] == 0) & (business_partial["cat_Landmark"] == 0)
business_partial[mask_non].shape

(17040, 44)

### Include sub category columns

In [106]:
mlb = MultiLabelBinarizer()

In [107]:
with open('sub_category/restaurantfood.pkl', 'rb') as f:
    sub_restaurantfood = pickle.load(f)

In [108]:
# keep only the terms in sub_category list

business_r_and_f['categories_clean'] = business_r_and_f['categories_clean'].apply(
                                        lambda x: [cat for cat in x if cat in sub_restaurantfood])

business_r_and_f['categories_clean']

C:\Users\mm4me\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


business_id
--9e1ONYQuAa-CB_Rrw7Tw                          [Steakhouses, Cajun/Creole]
--DaPTJW3-tB1vP-PfdTEg                                 [Breakfast & Brunch]
--KCl2FvVQpvjzmZSPyviA                                  [Sandwiches, Pizza]
--cZ6Hhc9F7VkKXxHMVZSQ                  [Spanish, Peruvian, Latin American]
-01XupAWZEXbdNbxNg5mEg          [Bars, American (Traditional), Sports Bars]
-0BxAGlIk5DJAGVkpqBXxg                                       [Coffee & Tea]
-0QtTRrAMn6DKLZNef3Ojg    [Delis, Specialty Food, Meat Shops, Ethnic Foo...
-0WegMt6Cy966qlDKhu6jA          [Burgers, Barbeque, American (Traditional)]
-0aOudcaAyac0VJbMX-L1g                              [Middle Eastern, Pizza]
-0tgMGl7D9B10YjSN2ujLA         [Pubs, Irish, Music Venues, Bars, Irish Pub]
-1UMR00eXtwaeh59pEiDjA    [Cocktail Bars, Breakfast & Brunch, Bars, Sand...
-1m9o3vGRA8IBPNvNqKLmA    [African, Bars, American (New), Cocktail Bars,...
-1uVyyy3UXNA9WoPucNniA    [Meat Shops, Grocery, Seafood Markets, Fruits ...


In [109]:
mlb.fit_transform(business_r_and_f["categories_clean"])
mlb.classes_

array(['Acai Bowls', 'Afghan', 'African', 'American (New)',
       'American (Traditional)', 'Arcades', 'Asian Fusion', 'Bagels',
       'Bakeries', 'Barbeque', 'Bars', 'Bed & Breakfast', 'Beer',
       'Beer Bar', 'Bistros', 'Brasseries', 'Brazilian',
       'Breakfast & Brunch', 'Breweries', 'British', 'Bubble Tea',
       'Buffets', 'Burgers', 'Butcher', 'Cafes', 'Cajun/Creole',
       'Canadian (New)', 'Candy Stores', 'Cantonese', 'Caribbean',
       'Casinos', 'Caterers', 'Cheesesteaks', 'Chicken Shop',
       'Chicken Wings', 'Chinese', 'Chocolatiers & Shops',
       'Cocktail Bars', 'Coffee & Tea', 'Coffee Roasteries',
       'Comfort Food', 'Convenience Stores', 'Creperies', 'Cupcakes',
       'Custom Cakes', 'Dance Clubs', 'Delicatessen', 'Delis',
       'Department Stores', 'Desserts', 'Dim Sum', 'Diners', 'Dive Bars',
       'Do-It-Yourself Food', 'Donairs', 'Donuts', 'Drugstores',
       'Ethiopian', 'Ethnic Food', 'Event Planning & Services', 'Falafel',
       'Farmers Mar

In [110]:
len(mlb.classes_)

162

In [111]:
sub_cat_df = pd.DataFrame(mlb.fit_transform(business_r_and_f["categories_clean"]),
                         columns=mlb.classes_,
                         index=business_r_and_f.index)

In [112]:
sub_cat_df

,Acai Bowls,Afghan,African,American (New),American (Traditional),Arcades,Asian Fusion,Bagels,Bakeries,Barbeque,...,Thai,Turkish,Vegan,Vegetarian,Venues & Event Spaces,Vietnamese,Waffles,Wine & Spirits,Wine Bars,Wraps
business_id,,,,,,,,,,,,,,,,,,,,,
--9e1ONYQuAa-CB_Rrw7Tw,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
--DaPTJW3-tB1vP-PfdTEg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
--KCl2FvVQpvjzmZSPyviA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
--cZ6Hhc9F7VkKXxHMVZSQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-01XupAWZEXbdNbxNg5mEg,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-0BxAGlIk5DJAGVkpqBXxg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-0QtTRrAMn6DKLZNef3Ojg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-0WegMt6Cy966qlDKhu6jA,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
-0aOudcaAyac0VJbMX-L1g,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
# save file
sub_cat_df.to_csv("sub_category_df.csv")

In [114]:
# concatenate
#business_r_and_f = pd.concat([business_r_and_f, sub_cat_df_norm], axis=1)
business_r_and_f = pd.concat([business_r_and_f, sub_cat_df], axis=1)
business_r_and_f.drop(columns=['categories', 'categories_clean'], inplace=True)

In [115]:
business_r_and_f.head()

,name,city,stars,price,review_count,1star_counts,2star_counts,3star_counts,4star_counts,5star_counts,...,Thai,Turkish,Vegan,Vegetarian,Venues & Event Spaces,Vietnamese,Waffles,Wine & Spirits,Wine Bars,Wraps
business_id,,,,,,,,,,,,,,,,,,,,,
--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,Las Vegas,4.0,4.0,1546,51.0,137.0,176.0,420.0,762.0,...,0,0,0,0,0,0,0,0,0,0
--DaPTJW3-tB1vP-PfdTEg,Sunnyside Grill,Toronto,3.5,2.0,45,4.0,3.0,9.0,16.0,13.0,...,0,0,0,0,0,0,0,0,0,0
--KCl2FvVQpvjzmZSPyviA,Hungry Howie's Pizza,Charlotte,3.0,1.0,13,3.0,2.0,3.0,1.0,4.0,...,0,0,0,0,0,0,0,0,0,0
--cZ6Hhc9F7VkKXxHMVZSQ,Pio Pio,Charlotte,4.0,2.0,333,22.0,22.0,43.0,111.0,135.0,...,0,0,0,0,0,0,0,0,0,0
-01XupAWZEXbdNbxNg5mEg,18 Degrees Neighborhood Grill,Scottsdale,3.0,2.0,82,20.0,14.0,17.0,22.0,9.0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
[col for col in business_r_and_f.columns]

['name',
 'city',
 'stars',
 'price',
 'review_count',
 '1star_counts',
 '2star_counts',
 '3star_counts',
 '4star_counts',
 '5star_counts',
 'text',
 'text_friendly',
 'text_reasonable',
 'text_local',
 'text_comfortable',
 'text_easy',
 'text_quick',
 'cat_Restaurants',
 'cat_Food',
 'cat_Shopping',
 'cat_Active Life',
 'cat_Arts & Entertainment',
 'cat_Nightlife',
 'cat_Hotels & Travel',
 'cat_Tours',
 'cat_Landmark',
 'cat_Others',
 'state',
 'address',
 'neighborhood',
 'postal_code',
 'latitude',
 'longitude',
 'monday',
 'tuesday',
 'wednesday',
 'thursday',
 'friday',
 'saturday',
 'sunday',
 'las_vegas',
 'word_count',
 'Acai Bowls',
 'Afghan',
 'African',
 'American (New)',
 'American (Traditional)',
 'Arcades',
 'Asian Fusion',
 'Bagels',
 'Bakeries',
 'Barbeque',
 'Bars',
 'Bed & Breakfast',
 'Beer',
 'Beer Bar',
 'Bistros',
 'Brasseries',
 'Brazilian',
 'Breakfast & Brunch',
 'Breweries',
 'British',
 'Bubble Tea',
 'Buffets',
 'Burgers',
 'Butcher',
 'Cafes',
 'Cajun/Creol

In [119]:
# keep only necessary columns and reorder
features = ['name',
 'city',
 'las_vegas',
 'stars',
 'price',
 'review_count',
 '1star_counts',
 '2star_counts',
 '3star_counts',
 '4star_counts',
 '5star_counts',
 'text',
 'word_count',
 'text_friendly',
 'text_reasonable',
 'text_local',
 'text_comfortable',
 'text_easy',
 'text_quick',
 'cat_Restaurants',
 'cat_Food',
 'cat_Shopping',
 'cat_Active Life',
 'cat_Arts & Entertainment',
 'cat_Nightlife',
 'cat_Hotels & Travel',
 'cat_Tours',
 'cat_Landmark',
 'cat_Others',
 'state',
 'address',
 'neighborhood',
 'postal_code',
 'latitude',
 'longitude',
 'monday',
 'tuesday',
 'wednesday',
 'thursday',
 'friday',
 'saturday',
 'sunday',
 'Acai Bowls',
 'Afghan',
 'African',
 'American (New)',
 'American (Traditional)',
 'Arcades',
 'Asian Fusion',
 'Bagels',
 'Bakeries',
 'Barbeque',
 'Bars',
 'Bed & Breakfast',
 'Beer',
 'Beer Bar',
 'Bistros',
 'Brasseries',
 'Brazilian',
 'Breakfast & Brunch',
 'Breweries',
 'British',
 'Bubble Tea',
 'Buffets',
 'Burgers',
 'Butcher',
 'Cafes',
 'Cajun/Creole',
 'Canadian (New)',
 'Candy Stores',
 'Cantonese',
 'Caribbean',
 'Casinos',
 'Caterers',
 'Cheesesteaks',
 'Chicken Shop',
 'Chicken Wings',
 'Chinese',
 'Chocolatiers & Shops',
 'Cocktail Bars',
 'Coffee & Tea',
 'Coffee Roasteries',
 'Comfort Food',
 'Convenience Stores',
 'Creperies',
 'Cupcakes',
 'Custom Cakes',
 'Dance Clubs',
 'Delicatessen',
 'Delis',
 'Department Stores',
 'Desserts',
 'Dim Sum',
 'Diners',
 'Dive Bars',
 'Do-It-Yourself Food',
 'Donairs',
 'Donuts',
 'Drugstores',
 'Ethiopian',
 'Ethnic Food',
 'Event Planning & Services',
 'Falafel',
 'Farmers Market',
 'Fashion',
 'Fast Food',
 'Filipino',
 'Fish & Chips',
 'Flowers & Gifts',
 'Food Court',
 'Food Delivery Services',
 'Food Stands',
 'Food Trucks',
 'French',
 'Fruits & Veggies',
 'Gastropubs',
 'Gelato',
 'German',
 'Gluten-Free',
 'Golf',
 'Greek',
 'Grocery',
 'Halal',
 'Hawaiian',
 'Health Markets',
 'Home & Garden',
 'Hookah Bars',
 'Hot Dogs',
 'Hot Pot',
 'Hotels',
 'Ice Cream & Frozen Yogurt',
 'Imported Food',
 'Indian',
 'International Grocery',
 'Internet Cafes',
 'Irish',
 'Irish Pub',
 'Italian',
 'Japanese',
 'Juice Bars & Smoothies',
 'Karaoke',
 'Kebab',
 'Korean',
 'Kosher',
 'Latin American',
 'Lebanese',
 'Local Flavor',
 'Lounges',
 'Meat Shops',
 'Mediterranean',
 'Mexican',
 'Middle Eastern',
 'Modern European',
 'Music Venues',
 'Noodles',
 'Organic Stores',
 'Pakistani',
 'Pan Asian',
 'Party & Event Planning',
 'Pasta Shops',
 'Patisserie/Cake Shop',
 'Persian/Iranian',
 'Peruvian',
 'Pizza',
 'Poke',
 'Polish',
 'Pool Halls',
 'Portuguese',
 'Poutineries',
 'Pubs',
 'Ramen',
 'Salad',
 'Salvadoran',
 'Sandwiches',
 'Seafood',
 'Seafood Markets',
 'Shaved Ice',
 'Shopping Centers',
 'Soul Food',
 'Soup',
 'Southern',
 'Spanish',
 'Specialty Food',
 'Sports Bars',
 'Steakhouses',
 'Street Vendors',
 'Sushi Bars',
 'Szechuan',
 'Tacos',
 'Taiwanese',
 'Tapas Bars',
 'Tapas/Small Plates',
 'Tea Rooms',
 'Tex-Mex',
 'Thai',
 'Turkish',
 'Vegan',
 'Vegetarian',
 'Venues & Event Spaces',
 'Vietnamese',
 'Waffles',
 'Wine & Spirits',
 'Wine Bars',
 'Wraps'
           ]

business_r_and_f = business_r_and_f[features]

In [120]:
business_r_and_f.shape

(18504, 204)

In [122]:
# save file
business_r_and_f.to_csv("business_r_and_f.csv")